In [ ]:
import os
import tqdm
import time
import pandas as pd
import numpy as np


In [ ]:
parent_dir = os.path.abspath('../')
parent_dir

In [ ]:
df = pd.read_csv(f"{parent_dir}/rummagenexrummageo.csv")

df = df['overlaps']
lst = set()
for ele in df:
    r = ele.split(";")
    lst.update(r)
with open("data/genes.txt", 'w') as file:
    file.writelines(f"{rummageo}\n" for rummageo in lst)
with open("data/genes.txt", 'r') as file:
    lines = file.readlines()

lst = set([line.strip() for line in lines])




In [ ]:
from Bio import Entrez
Entrez.email = os.getenv('EMAIL')
Entrez.api_key = os.getenv('API_KEY')
from tqdm import tqdm



def get_all_genes(path, name):
    lst = set()
    nom = set()
    k = pd.read_csv(f'data/pubmed_rif_{name}.csv')
    nom = set(k["gene"])
    for ele in path:
        r = ele.split(";")
        lst.update(r)
    print(len(lst))
    lst = lst.difference(nom)
    print(len(lst))
    return lst


def find_citations(genes, name):
   db = 'pubmed'
   domain = 'https://www.ncbi.nlm.nih.gov/entrez/eutils'
   retmode='json'


   with open(f'data/pubmed_rif_{name}.csv', 'a') as f:
       for g in tqdm(genes, desc="Processing"):
           query = g
           handle = Entrez.esearch(db="pubmed", retmax=9999999 ,term=g, idtype="acc")
           record = Entrez.read(handle)
           handle.close()
           f.write(f"{g},{len(record['IdList'])}\n")
           time.sleep(1)
   print("done")

genes = get_all_genes(lst, "all")


In [ ]:
find_citations(genes, "all")

In [ ]:
df = pd.read_csv(f"{parent_dir}/rummagenexrummageo.csv")
df = df.sort_values(by=["p-value", "odds"], ascending=[True, False])
df = df.head(1000000)
df



In [ ]:
from collections import Counter

def make_words_df(df, species):
    if species != "all":
        dfe = df.loc[df["species"] == species]
    else:
        dfe = df
    split_lists = dfe['overlaps'].str.split(';')
    flattened_list = [item for sublist in split_lists for item in sublist]

    word_counts = Counter(flattened_list)
    word_counts_df = pd.DataFrame(word_counts.items(), columns=['gene', 'count_file'])
    word_counts_df.to_csv(f"data/all_word_df.csv", index=False)
make_words_df(df, "all")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
import numpy as np
from matplotlib import colors as mcolors

In [ ]:
import numpy as np
from scipy.spatial import distance_matrix


df1 = pd.read_csv("data/all_word_df.csv")
df2 = pd.read_csv("data/pubmed_rif_all.csv")
lala = pd.merge(df1, df2, on="gene")

# Panel a: Scatter plot of Set Appearances vs Citations
plt.figure(figsize=(10, 8), dpi=300)
column_x = lala["count_file"]
column_y = lala["citations"]
gene = lala["gene"]
plt.scatter(column_x, column_y, alpha=0.5, color="black")

plt.xlabel('Set Appearances', fontsize=24)
plt.ylabel('Citations', fontsize=24)
plt.xscale('log')
plt.yticks(fontsize=18)
plt.xticks(fontsize=18)
plt.tight_layout()
plt.savefig("figures/gene_count_citation2.png", dpi=300)
plt.savefig("figures/gene_count_citation2.pdf", dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(10, 8), dpi=300)
numbers = df["n-overlap"]
plt.hist(numbers, bins=50, color="black")
plt.xlabel('Gene Set Length', fontsize=24)
plt.ylabel('Frequency', fontsize=24)

plt.yticks(fontsize=18)
plt.xticks(fontsize=18)
plt.tight_layout()
plt.savefig("figures/gene_length.png", dpi=300)
plt.savefig("figures/gene_length.pdf", dpi=300)
plt.show()


In [ ]:
print(df["n-overlap"].mean())
print(df["n-overlap"].median())
print(df["n-overlap"].mode())
print(df["n-overlap"].std())
print(df["n-overlap"].max())
print(df["n-overlap"].min())